# Multi-Document Agentic RAG for Quantum Computing 

## Setup OPENAI API KEY

If you don't have one, get it [here](https://platform.openai.com/account/api-keys)

In [1]:
OPENAI_API_KEY = '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'
import openai
openai.api_key = OPENAI_API_KEY

## Imports and utils

In [2]:
# imports 

from llama_index.core import download_loader
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.tools import QueryEngineTool, FunctionTool
from llama_index.core.objects import ObjectIndex

from pathlib import Path
from typing import List, Dict

import nest_asyncio
nest_asyncio.apply()

In [4]:


# util functions 
def get_vector_tool(nodes:List,algo:str):
    '''creates a vector index tool that performs vector search'''

    vector_index = VectorStoreIndex(nodes)
    def vector_query(query:str) -> str:
        """Perform a vector search over an index.
    
        query (str): the string query to be embedded.
        """
        query_engine = vector_index.as_query_engine(similarity_top_k=4)
        response = query_engine.query(query)
        return response
    vector_query_tool = FunctionTool.from_defaults(
    name=f"{algo}_vector_tool",
    fn=vector_query
    )

    return vector_query_tool

def get_summary_tool(nodes:List, algo:str,llm:any):
    '''creates a summary index tool that performs summarization'''
    
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
        llm=llm
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"{algo}_summary_tool",
        query_engine=summary_query_engine,
        description=(
            f"Useful if you want to get a summary or explanation of {algo} algorithm"
        ),
    )

    return summary_tool

def get_tools(doc_paths:Dict, loader:any,llm:any) -> Dict:
    '''returns vector index and summary index tools for the provided jupyter notebooks'''

    algo_to_tools_dict = {}
    for algo, doc_path in doc_paths.items():

        # split text into chunks
        document = loader.load_data(file=Path(doc_path))
        splitter = TokenTextSplitter(
            chunk_size=64,
            chunk_overlap=10,
            separator=" ",
        )
        nodes = splitter.get_nodes_from_documents(document)
        if len(nodes)<=1:
            raise ValueError('Number of generated nodes is less than or equal to 1. Check the document at {doc_path}')
        
        # get summary and vector index tool
        vector_tool = get_vector_tool(nodes=nodes, algo=algo)
        summary_tool = get_summary_tool(nodes=nodes, algo=algo,llm=llm)

        # store tools for the concerned algorithm
        algo_to_tools_dict[algo] = [vector_tool, summary_tool]
    
    algos = list(doc_paths.keys())
    tools = [tool for algo in algos for tool in algo_to_tools_dict[algo]]

    return tools



## Implementation

In [5]:
# Download and initialize the IPYNBReader loader
IPYNBReader = download_loader("IPYNBReader")
loader = IPYNBReader(concatenate=True)

/var/folders/2y/wlq7_0_s2bq1rwtt21r_h35c0000gn/T/ipykernel_46514/3078635327.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  IPYNBReader = download_loader("IPYNBReader")


In [6]:
doc_paths = {
    "bernstein-vazirani": r"/Users/nihaalyk/Downloads/data/bernstein-vazirani algorithm.ipynb",
    "deutsch-jozsa":r"/Users/nihaalyk/Downloads/data/deutsch-jozsa algorithm.ipynb",
    "grover":r"/Users/nihaalyk/Downloads/data/grover's algorithm.ipynb",
    "quantum-phase-estimation":r"/Users/nihaalyk/Downloads/data/quantum-phase-estimation algorithm.ipynb",
    "shor":r"/Users/nihaalyk/Downloads/data/shor's algorithm.ipynb",
    "simon":r"/Users/nihaalyk/Downloads/data/simon's algorithm.ipynb"
}

In [9]:
# Import the Groq client
from groq import Groq

# Initialize the Groq client
client = Groq()

# Define the model and parameters for the completion
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[],  # Add your messages here
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Stream the completion results
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: gsk_BvDQ********************************************1z8Q. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [7]:
# initialize agents
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever, 
    llm=model, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [8]:
# query agent
response = agent.query(
    "Tell me about Quantum Circuits"
)

Added user message to memory: Tell me about Quantum Circuits


=== Calling Function ===
Calling function: quantum-phase-estimation_summary_tool with args: {"input": "Quantum Circuits"}
=== Function Output ===
Quantum circuits are constructed using quantum gates that operate on qubits. These circuits are fundamental in quantum computing and are used to perform various quantum algorithms and operations. Quantum circuits are built by arranging quantum gates in a specific order to manipulate the quantum states of qubits. The gates can include operations like Hadamard gates, controlled gates, phase gates, and more. The circuits are designed to perform tasks such as quantum phase estimation, quantum Fourier transformation, and other quantum algorithms. By executing these circuits on quantum computers, we can explore quantum phenomena and solve complex problems efficiently in the quantum realm.
=== LLM Response ===
Quantum circuits are fundamental in quantum computing and are constructed using quantum gates that operate on qubits. These circuits are used

## References

[Building Agentic RAG with LlamaIndex](https://www.deeplearning.ai/short-courses/building-agentic-rag-with-llamaindex/)